In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

/home/nxie/code/nikki099/de_linkedin_etl_project/.venv/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [3]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
# df_daily_all

Data Engineer (16, 51)
Data Scientist (2, 45)
Data Analyst (11, 51)


In [4]:
# df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,seniority,directapply,linkedin_org_slug,salary_raw.@type,salary_raw.currency,salary_raw.value.@type,salary_raw.value.minValue,salary_raw.value.maxValue,salary_raw.value.unitText,job_category
0,1638778785,2025-05-09T05:18:10,2025-05-09T05:33:52.028966,Data Engineer - Amazon FinTech,Amazon,https://www.linkedin.com/company/amazon,2025-06-08T05:18:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Not Applicable,False,amazon,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
1,1638715102,2025-05-09T04:54:51,2025-05-09T05:04:01.754756,AWS Data Engineer,DR Analytics Recruitment,https://www.linkedin.com/company/dr-analytics-...,2025-06-08T04:54:51,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Not Applicable,False,dr-analytics-recruitment,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
2,1638624041,2025-05-09T04:08:30,2025-05-09T04:22:32.081616,Data Engineer,Avance Consulting,https://www.linkedin.com/company/avance-services,2025-06-08T04:08:30,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,avance-services,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
3,1638430873,2025-05-09T02:38:33,2025-05-09T02:50:45.770691,Senior Software Engineer - Enterprise Data Col...,Nuix,https://www.linkedin.com/company/nuix,2025-06-08T02:38:32,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,nuix,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
4,1638321622,2025-05-09T01:49:25,2025-05-09T02:02:41.672398,🌟 DevOps Engineer - MedTech Data Platform Scal...,Hyre.,https://www.linkedin.com/company/hyre.,2025-06-08T01:49:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,hyre.,MonetaryAmount,AUD,QuantitativeValue,140000.0,180000.0,YEAR,Data Engineer


In [5]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['title'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(22, 51)

In [6]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [7]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(22, 51)

In [8]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1638778785,2025-05-09T05:18:10,2025-05-09T05:33:52.028966,Data Engineer - Amazon FinTech,Data Engineer,Amazon,https://www.linkedin.com/company/amazon,2025-06-08T05:18:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Seattle, WA",Public Company,NaN,"['e-Commerce', 'Retail', 'Operations', 'and In...","['2127 7th Ave., Seattle, WA 98109, US', '1290...",Amazon is guided by four principles: customer ...,False,Not Applicable,False,amazon
1,1638715102,2025-05-09T04:54:51,2025-05-09T05:04:01.754756,AWS Data Engineer,Data Engineer,DR Analytics Recruitment,https://www.linkedin.com/company/dr-analytics-...,2025-06-08T04:54:51,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Perth, WA",Self-Owned,2023.0,[''],"['Perth, WA 6000, AU']",DR Analytics Recruitment helps businesses hire...,True,Not Applicable,False,dr-analytics-recruitment


### Process the df_daily_all data 


In [9]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-09
1    2025-05-09
2    2025-05-09
3    2025-05-09
4    2025-05-09
Name: job_date, dtype: object

In [10]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1638778785,2025-05-09T05:18:10,2025-05-09T05:33:52.028966,Data Engineer - Amazon FinTech,Data Engineer,Amazon,https://www.linkedin.com/company/amazon,2025-06-08T05:18:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['e-Commerce', 'Retail', 'Operations', 'and In...","['2127 7th Ave., Seattle, WA 98109, US', '1290...",Amazon is guided by four principles: customer ...,False,Not Applicable,False,amazon,2025-05-09,Sydney,NSW
1,1638715102,2025-05-09T04:54:51,2025-05-09T05:04:01.754756,AWS Data Engineer,Data Engineer,DR Analytics Recruitment,https://www.linkedin.com/company/dr-analytics-...,2025-06-08T04:54:51,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['Perth, WA 6000, AU']",DR Analytics Recruitment helps businesses hire...,True,Not Applicable,False,dr-analytics-recruitment,2025-05-09,Perth,WA
2,1638624041,2025-05-09T04:08:30,2025-05-09T04:22:32.081616,Data Engineer,Data Engineer,Avance Consulting,https://www.linkedin.com/company/avance-services,2025-06-08T04:08:30,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT Services', 'Staffing and Recruiting', 'Hu...","['Office No 301C, Golden Cross House, 8 Duncan...",Avance Consulting is a global organization foc...,True,Mid-Senior level,True,avance-services,2025-05-09,Melbourne,VIC
3,1638162209,2025-05-09T00:49:12,2025-05-09T01:03:40.711737,Senior Data Engineer,Data Engineer,Tranzformd,https://www.linkedin.com/company/tranzformd,2025-06-08T00:48:33,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Business Transformation', 'BI & Data Analyti...","['7 Macquarie Place, Level 3, Sydney, New Sout...",Tranzformd is a management consulting and prog...,False,Mid-Senior level,False,tranzformd,2025-05-09,Sydney,NSW
4,1638053647,2025-05-09T00:07:06,2025-05-09T00:22:19.448588,Neo4j / AWS Neptune - Graph Data Engineer / T...,Data Engineer,Private Advertiser,https://www.linkedin.com/company/private-adver...,2025-06-08T00:07:06,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Mining', 'Business Development', 'and Sales']",[],Based in Mackay and with locations situated in...,False,Senior medewerker,True,private-advertiser,2025-05-09,Sydney,NSW


In [11]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

17     FULL_TIME
18     FULL_TIME
19     FULL_TIME
20     FULL_TIME
21    CONTRACTOR
Name: employment_type, dtype: object

In [12]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

17           11-50
18       501-1,000
19         201-500
20          51-200
21    5,001-10,000
Name: org_size, dtype: object

In [13]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1638778785,2025-05-09T05:18:10,2025-05-09T05:33:52.028966,Data Engineer - Amazon FinTech,Data Engineer,Amazon,https://www.linkedin.com/company/amazon,2025-06-08T05:18:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Amazon is guided by four principles: customer ...,False,Not Applicable,False,amazon,2025-05-09,Sydney,NSW,FULL_TIME,"10,001+"
1,1638715102,2025-05-09T04:54:51,2025-05-09T05:04:01.754756,AWS Data Engineer,Data Engineer,DR Analytics Recruitment,https://www.linkedin.com/company/dr-analytics-...,2025-06-08T04:54:51,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,DR Analytics Recruitment helps businesses hire...,True,Not Applicable,False,dr-analytics-recruitment,2025-05-09,Perth,WA,FULL_TIME,2-10


In [14]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1638778785,Data Engineer - Amazon FinTech,Data Engineer,2025-05-09,Sydney,NSW,FULL_TIME,Amazon,https://www.linkedin.com/company/amazon,https://au.linkedin.com/jobs/view/data-enginee...,...,"Seattle, WA",Public Company,NaN,"['e-Commerce', 'Retail', 'Operations', 'and In...","['2127 7th Ave., Seattle, WA 98109, US', '1290...",Amazon is guided by four principles: customer ...,False,Not Applicable,False,amazon
1,1638715102,AWS Data Engineer,Data Engineer,2025-05-09,Perth,WA,FULL_TIME,DR Analytics Recruitment,https://www.linkedin.com/company/dr-analytics-...,https://au.linkedin.com/jobs/view/aws-data-eng...,...,"Perth, WA",Self-Owned,2023.0,[''],"['Perth, WA 6000, AU']",DR Analytics Recruitment helps businesses hire...,True,Not Applicable,False,dr-analytics-recruitment
2,1638624041,Data Engineer,Data Engineer,2025-05-09,Melbourne,VIC,CONTRACTOR,Avance Consulting,https://www.linkedin.com/company/avance-services,https://au.linkedin.com/jobs/view/data-enginee...,...,"London, United Kingdom",Privately Held,2007.0,"['IT Services', 'Staffing and Recruiting', 'Hu...","['Office No 301C, Golden Cross House, 8 Duncan...",Avance Consulting is a global organization foc...,True,Mid-Senior level,True,avance-services
3,1638162209,Senior Data Engineer,Data Engineer,2025-05-09,Sydney,NSW,FULL_TIME,Tranzformd,https://www.linkedin.com/company/tranzformd,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, New South Wales",Privately Held,2021.0,"['Business Transformation', 'BI & Data Analyti...","['7 Macquarie Place, Level 3, Sydney, New Sout...",Tranzformd is a management consulting and prog...,False,Mid-Senior level,False,tranzformd
4,1638053647,Neo4j / AWS Neptune - Graph Data Engineer / T...,Data Engineer,2025-05-09,Sydney,NSW,FULL_TIME,Private Advertiser,https://www.linkedin.com/company/private-adver...,https://au.linkedin.com/jobs/view/neo4j-aws-ne...,...,NaN,Self-Owned,NaN,"['Mining', 'Business Development', 'and Sales']",[],Based in Mackay and with locations situated in...,False,Senior medewerker,True,private-advertiser


## Connect to  Snowflake database for raw data initial processing

In [15]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [16]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_6147/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(356, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [17]:
df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [18]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [19]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

15 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
10,1638625246,Mission Data Analyst,Data Analyst,2025-05-09,East Melbourne,VIC,FULL_TIME,Melbourne Archdiocese Catholic Schools,https://www.linkedin.com/company/melbourne-arc...,https://au.linkedin.com/jobs/view/mission-data...,...,"East Melbourne, Victoria",Educational,2020.0,[''],"['228 Victoria Pde, East Melbourne, Victoria 3...","290 schools. 118,300 students. 18,860 staff.\n...",False,Tidak Berlaku,False,melbourne-archdiocese-catholic-schools
11,1638143591,Data Analyst - Superannuation & Retirement Str...,Data Analyst,2025-05-09,Sydney,NSW,FULL_TIME,Parity Consulting,https://www.linkedin.com/company/parity-consul...,https://au.linkedin.com/jobs/view/data-analyst...,...,"SYDNEY, NSW",Privately Held,2012.0,"['Product Management', 'Marketing', 'Superannu...","['Level 12, 35 Clarence St, SYDNEY, NSW 2000, ...",Connecting Business Leaders with Top-Tier Tale...,True,Entry level,True,parity-consulting
12,1638126312,"Senior Data Analyst, Roll Data and Client Enga...",Data Analyst,2025-05-09,Canberra,None,FULL_TIME,Australian Electoral Commission,https://www.linkedin.com/company/australian-el...,https://au.linkedin.com/jobs/view/senior-data-...,...,NaN,Government Agency,1984.0,['Electoral adminstration and Supply chain'],"['', '10 Mort St, City, Australian Capital Ter...",The AEC has been around since 1984 and prior t...,False,Not Applicable,False,australian-electoral-commission
13,1636279823,Data Analyst and Power Platform Developer,Data Analyst,2025-05-08,Brisbane,QLD,FULL_TIME,GR Production Services,https://www.linkedin.com/company/gr-production...,https://au.linkedin.com/jobs/view/data-analyst...,...,"Perth, Western Australia",Privately Held,1997.0,"['Operations & Maintenance', 'Wellsites', 'Eng...","['99 St Georges Tce, Level 10, Perth, Western ...",GR Production Services (GRPS) operate and main...,False,Mid-Senior level,False,gr-production-services
14,1638372301,Data Analyst,Data Analyst,2025-05-09,Sydney,NSW,CONTRACTOR,Hays,https://www.linkedin.com/company/hays,https://au.linkedin.com/jobs/view/data-analyst...,...,"London, England",Public Company,NaN,"['Recruitment', 'Staffing solutions', 'Resourc...","['20 Triton Street, London, England NW1 3BF, G...",We are leaders in specialist recruitment and w...,True,Mid-Senior level,True,hays


In [20]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

def translate_text(text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        translated_text = translator.translate(str(text), dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
    unique_values = df_new_jobs[col].dropna().unique()
    translation_map = {val: translate_text(val, target_language='en') for val in unique_values}
    df_new_jobs[col] = df_new_jobs[col].map(translation_map).fillna('NA')

In [21]:
df_new_jobs['CITY'].unique()

array(['Sydney', 'Perth', 'Melbourne', 'Woden', 'East Melbourne',
       'Canberra', 'Brisbane'], dtype=object)

In [22]:
df_new_jobs['STATE'].unique()

array(['NSW', 'Of', 'VIC', 'NA', 'QLD'], dtype=object)

In [23]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
